# Desafio de Dados: Consumindo API de Aluguel de Bicicleta

## Resolução 

In [31]:
from pyspark.sql import SparkSession
import requests
from pyspark.sql.functions import date_format
from pyspark.sql.functions import from_unixtime, unix_timestamp, avg,hour
from pyspark.sql.functions import col

1-Obter a lista de todas as estações de bicicletas e armazená-las em uma tabela SQL.

In [32]:

# Sessão Spark
spark = SparkSession.builder.appName("EstacoesDeBicicletas").getOrCreate()

# URL com informações sobre as estações
url = "https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json"

response = requests.get(url)

# Verifica se a solicitação foi bem-sucedida
if response.status_code == 200:
    data = response.json()  # Os dados são convertidos em um dicionário Python

    # Transforma os dados em um DataFrame Spark
    df = spark.createDataFrame(data['data']['stations'])

    # Seleciona apenas  as colunas 'capacity', 'name' e 'station_id'
    df_estacoes = df.select('capacity', 'name', 'station_id')

    # Cria tabela temporária a partir do DataFrame
    df_estacoes.createOrReplaceTempView("estacoes_bikes")

    # Exiba o DataFrame com as informações das estações
    df_estacoes.show()

   
else:
    print("Falha ao obter os dados. Código de status:", response.status_code)


+--------+--------------------+--------------------+
|capacity|                name|          station_id|
+--------+--------------------+--------------------+
|      87|  E 40 St & Park Ave|c638ec67-9ac0-416...|
|       0|Bailey Ave & W 22...| 1862788202397235246|
|       0|Crotona Ave & E 1...| 1860189327773878824|
|       0|    E 184 St & 3 Ave| 1864605386632012480|
|       0|Crotona Pkwy & Br...| 1861678548643203686|
|       0|Arthur Ave & E 18...| 1862008701134683118|
|       0|Sedgwick Ave & W ...| 1869753484503987044|
|      37|Columbia St & Lor...|66de0cab-0aca-11e...|
|       0|E 189 St & Bathga...| 1862008939006380034|
|      25|      56 Ave & 92 St| 1846086362824884662|
|      20|Decatur Ave & Bed...|6542d952-ca19-410...|
|      19|      64 St & 51 Ave|5145ad1c-241f-444...|
|      96|     8 Ave & W 33 St|66dc686c-0aca-11e...|
|      19|      37 St & 24 Ave|0630217a-3913-47d...|
|      19|Wilkins Ave & Cro...|068b919c-f649-409...|
|      19|E 163 St & Tiffan...|6476c5fd-611e-4

In [33]:
# Executar uma consulta SQL na tabela temporária "estacoes_status_bicicletas" para selecionar as estações com um número mínimo de bicicletas disponíveis
resultado = spark.sql("SELECT * FROM estacoes_bikes WHERE capacity >= 87")
resultado.show()


+--------+--------------------+--------------------+
|capacity|                name|          station_id|
+--------+--------------------+--------------------+
|      87|  E 40 St & Park Ave|c638ec67-9ac0-416...|
|      96|     8 Ave & W 33 St|66dc686c-0aca-11e...|
|     106|    W 59 St & 10 Ave|66dc0dab-0aca-11e...|
|     103|  Broadway & W 48 St|64f0f28c-bedc-42d...|
|      95|      E 6 St & Ave B|66db76a1-0aca-11e...|
|      94|      1 Ave & E 6 St|c37931bb-8571-467...|
|     107|  Broadway & W 25 St|daefc84c-1b16-422...|
|     123| FDR Drive & E 35 St|66dc7659-0aca-11e...|
|      87|     E 27 St & 1 Ave|66dc9223-0aca-11e...|
|     120|Gansevoort St & H...| 1827839088308194240|
|      97|     8 Ave & W 31 St|66ddbd20-0aca-11e...|
|     117|Allen St & Hester St|66dbca9c-0aca-11e...|
|     105|Lafayette St & E ...|66db65aa-0aca-11e...|
|     117|    W 43 St & 10 Ave|66dc7de9-0aca-11e...|
|     107|     E 10 St & Ave A|66dc1beb-0aca-11e...|
|      87|     1 Ave & E 44 St|66dc2172-0aca-1

02-Identificar a estação de bicicleta com a maior quantidade de bicicletas disponíveis em tempo real e exibir o resultado em uma tabela SQL.

In [34]:

# Sessão Spark
spark = SparkSession.builder.appName("StatusBike").getOrCreate()

# URL com informações sobre as estações
url = "https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_status.json"

response = requests.get(url)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    data = response.json()  # Os dados são convertidos em um dicionário Python

    # Transforma os dados em um DataFrame Spark
    df = spark.createDataFrame(data['data']['stations'])
    # Seleciona apneas as colunas 'num_bikes_available' e 'station_id'
    df_status = df.select('num_bikes_available', 'station_id')

    # Cria uma tabela temporária
    df_status.createOrReplaceTempView("status_bikes")

    # Exibe com as informações das estações
    df_status.show(30)
else:
    print("Falha ao obter os dados. Código de status:", response.status_code)


+-------------------+--------------------+
|num_bikes_available|          station_id|
+-------------------+--------------------+
|                  0|c638ec67-9ac0-416...|
|                  0| 1862788202397235246|
|                  0| 1860189327773878824|
|                  0| 1864605386632012480|
|                  0| 1861678548643203686|
|                  0| 1862008701134683118|
|                  0| 1869753484503987044|
|                  0|66de0cab-0aca-11e...|
|                  0| 1862008939006380034|
|                  0| 1846086362824884662|
|                 17|6542d952-ca19-410...|
|                  8|5145ad1c-241f-444...|
|                 21|66dc686c-0aca-11e...|
|                 12|0630217a-3913-47d...|
|                  4|068b919c-f649-409...|
|                  2|6476c5fd-611e-436...|
|                  5|66dcc0b2-0aca-11e...|
|                  5|bcb08532-f0d5-4c0...|
|                 23|ca7ec509-cefc-4ac...|
|                  2|d18512f9-52c8-417...|
|          

In [35]:
# Executa consulta SQL na tabela temporária "estacoes_status_bicicletas"
resultado = spark.sql("SELECT * FROM status_bikes WHERE num_bikes_available >= 10")
resultado.show()


+-------------------+--------------------+
|num_bikes_available|          station_id|
+-------------------+--------------------+
|                 17|6542d952-ca19-410...|
|                 21|66dc686c-0aca-11e...|
|                 12|0630217a-3913-47d...|
|                 23|ca7ec509-cefc-4ac...|
|                 10| 1809664497883596992|
|                 15|c2bb1874-bcb7-47c...|
|                 11|7938263e-e5b8-4ca...|
|                 20|c80c55c4-d623-42d...|
|                 16|cc5f0e68-dd5f-4db...|
|                 16| 1854538962214948918|
|                 12|d11b9869-8964-4b7...|
|                 10|2c8b263e-952c-494...|
|                 15|7a1ad7c0-4958-435...|
|                 10|66de71af-0aca-11e...|
|                 13| 1835246937517543544|
|                 11|b442a648-e9f4-489...|
|                 18|66ddeaff-0aca-11e...|
|                 10|823ef531-40e6-439...|
|                 10| 1846085439406916338|
|                 21|66dc3782-0aca-11e...|
+----------

In [36]:
df_joined = spark.sql("""
    SELECT a.name AS nome_estacao, b.num_bikes_available
    FROM estacoes_bikes a
    INNER JOIN status_bikes b ON a.station_id = b.station_id
    ORDER BY b.num_bikes_available DESC
    LIMIT 1
""")
df_joined.show()

+--------------------+-------------------+
|        nome_estacao|num_bikes_available|
+--------------------+-------------------+
|Allen St & Hester St|                 97|
+--------------------+-------------------+



In [37]:
spark = SparkSession.builder.appName("dadosviagemSpark").getOrCreate()
arquivo = "202309-citibike-tripdata.csv"
df_dados = spark.read.csv(caminho_arquivo, header=True, inferSchema=True)

In [38]:
df_dados.show()

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|         start_lat|         start_lng|           end_lat|           end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+
|B0A0F1DEFA4B72FC|electric_bike|2023-09-03 10:20:41|2023-09-03 10:24:16|     E 1 St & Bowery|         5636.13|     E 10 St & 2 Ave|       5746.02| 40.72486122254819|-73.99213135242462| 40.72970805644994| -73.9865979552269|       member|
|2B26AB15647BF4EE| classic_bike|2023-09-27 15:44:23|

In [39]:
 # Seleciona as colunas 'num_bikes_available' e 'station_id'
df_dados = df_dados.select('start_station_name','started_at','end_station_name','ended_at')
df_dados.show()

+--------------------+-------------------+--------------------+-------------------+
|  start_station_name|         started_at|    end_station_name|           ended_at|
+--------------------+-------------------+--------------------+-------------------+
|     E 1 St & Bowery|2023-09-03 10:20:41|     E 10 St & 2 Ave|2023-09-03 10:24:16|
|Pearl St & Hanove...|2023-09-27 15:44:23|Allen St & Riving...|2023-09-27 15:53:25|
|     E 1 St & Bowery|2023-09-19 13:40:48|     E 10 St & 2 Ave|2023-09-19 13:48:11|
|Central Ave & Him...|2023-09-30 16:27:50| Mott St & Prince St|2023-09-30 16:56:35|
| St Marks Pl & 2 Ave|2023-09-21 16:59:53| Mott St & Prince St|2023-09-21 17:07:36|
|Meserole Ave & Ma...|2023-09-11 14:50:19|Franklin St & Dup...|2023-09-11 14:54:47|
|Pearl St & Hanove...|2023-09-06 17:19:49|Fulton St & Willi...|2023-09-06 17:23:40|
|Adams St & Prospe...|2023-09-26 17:48:10|Washington Ave & ...|2023-09-26 18:04:14|
|West Broadway & W...|2023-09-15 13:14:38|Allen St & Riving...|2023-09-15 14

In [40]:
#Renomeado as colunas
df_dados= df_dados.withColumnRenamed("start_station_name", "nome_estacao_inicio")\
               .withColumnRenamed("started_at", "inicio_em")\
               .withColumnRenamed("end_station_name", "nome_estacao_termino")\
               .withColumnRenamed("ended_at", "termino_em")
# Cria uma tabela temporária
df_dados.createOrReplaceTempView("dados_viagem")
df_dados.show()

+--------------------+-------------------+--------------------+-------------------+
| nome_estacao_inicio|          inicio_em|nome_estacao_termino|         termino_em|
+--------------------+-------------------+--------------------+-------------------+
|     E 1 St & Bowery|2023-09-03 10:20:41|     E 10 St & 2 Ave|2023-09-03 10:24:16|
|Pearl St & Hanove...|2023-09-27 15:44:23|Allen St & Riving...|2023-09-27 15:53:25|
|     E 1 St & Bowery|2023-09-19 13:40:48|     E 10 St & 2 Ave|2023-09-19 13:48:11|
|Central Ave & Him...|2023-09-30 16:27:50| Mott St & Prince St|2023-09-30 16:56:35|
| St Marks Pl & 2 Ave|2023-09-21 16:59:53| Mott St & Prince St|2023-09-21 17:07:36|
|Meserole Ave & Ma...|2023-09-11 14:50:19|Franklin St & Dup...|2023-09-11 14:54:47|
|Pearl St & Hanove...|2023-09-06 17:19:49|Fulton St & Willi...|2023-09-06 17:23:40|
|Adams St & Prospe...|2023-09-26 17:48:10|Washington Ave & ...|2023-09-26 18:04:14|
|West Broadway & W...|2023-09-15 13:14:38|Allen St & Riving...|2023-09-15 14

03- Calcular a média de duração de uma viagem em cada estação de bicicleta e exibir os resultados em uma tabela SQL.


In [41]:

df_dadosAvg = df_dados.groupBy("nome_estacao_inicio").agg(
    from_unixtime(avg(unix_timestamp("termino_em") - unix_timestamp("inicio_em")), "HH:mm:ss").alias("avg_duracao_viagem")
)
# Cria uma tabela temporária
df_dadosAvg.createOrReplaceTempView("avg_viagem")
# Consulta da tabela temporária avg_viagem
consulta_sql = "SELECT * FROM avg_viagem"
resultado = spark.sql(consulta_sql)
resultado.show(30)

#df_dados2.show()

+--------------------+------------------+
| nome_estacao_inicio|avg_duracao_viagem|
+--------------------+------------------+
|Nassau St & Duffi...|          00:14:07|
|Buchanan Pl & Gra...|          00:13:56|
|     E 63 St & 3 Ave|          00:18:25|
|       19 St & 8 Ave|          00:12:31|
|       45 Rd & 11 St|          00:14:33|
| Sterling Pl & 5 Ave|          00:14:24|
|  N 11 St & Kent Ave|          00:13:38|
|E Tremont Ave & E...|          00:20:06|
|College Ave & E 1...|          00:15:12|
|E 141 St & Jackso...|          00:13:43|
|       5 Ave & 66 St|          00:19:09|
|     31 Ave & 103 St|          00:17:12|
|       10 St & 5 Ave|          00:13:58|
|   Dean St & Hoyt St|          00:13:57|
|     W 20 St & 5 Ave|          00:14:25|
| Park Ave & E 124 St|          00:14:25|
|Popham Ave & W 17...|          00:16:41|
| 20 Ave & Shore Blvd|          00:17:01|
|Van Buren St & Br...|          00:14:35|
| Berkeley Pl & 6 Ave|          00:14:29|
|Grand Concourse &...|          00

 04-Calcular a média de duração de uma viagem em cada hora do dia e exibir os resultados em uma tabela SQL.

In [42]:
# Consulta para calcular a média de duração de viagem em cada hora do dia
query_avg_duration = """
SELECT
    DATE_FORMAT(CAST(inicio_em AS TIMESTAMP), 'HH') AS hora,
    FROM_UNIXTIME(AVG(UNIX_TIMESTAMP(termino_em) - UNIX_TIMESTAMP(inicio_em)), 'HH:mm:ss') AS media_duracao
FROM dados_viagem
GROUP BY hora
ORDER BY hora
"""

# Executa a consulta
resultado_media_duracao_hora = spark.sql(query_avg_duration)
resultado_media_duracao_hora.show(24)


+----+-------------+
|hora|media_duracao|
+----+-------------+
|  00|     00:17:06|
|  01|     00:17:35|
|  02|     00:17:56|
|  03|     00:19:35|
|  04|     00:17:30|
|  05|     00:13:35|
|  06|     00:13:41|
|  07|     00:14:24|
|  08|     00:14:36|
|  09|     00:14:38|
|  10|     00:16:00|
|  11|     00:16:41|
|  12|     00:16:51|
|  13|     00:16:55|
|  14|     00:16:49|
|  15|     00:16:35|
|  16|     00:16:25|
|  17|     00:15:56|
|  18|     00:15:40|
|  19|     00:15:22|
|  20|     00:15:14|
|  21|     00:15:19|
|  22|     00:16:17|
|  23|     00:16:28|
+----+-------------+



05-Obter o histórico de viagens de uma estação de bicicleta específica e exibir os resultados em uma tabela SQL.

In [43]:
# Filtra o Df para obter o histórico de viagens da estação 
historico_estacaoX = df_dados.filter((col("nome_estacao_inicio") == "E 1 St & Bowery") | (col("nome_estacao_termino") == "E 1 St & Bowery"))

# Exibe histórico de viagens 
historico_estacaoX.show()

+-------------------+-------------------+--------------------+-------------------+
|nome_estacao_inicio|          inicio_em|nome_estacao_termino|         termino_em|
+-------------------+-------------------+--------------------+-------------------+
|    E 1 St & Bowery|2023-09-03 10:20:41|     E 10 St & 2 Ave|2023-09-03 10:24:16|
|    E 1 St & Bowery|2023-09-19 13:40:48|     E 10 St & 2 Ave|2023-09-19 13:48:11|
|    E 1 St & Bowery|2023-09-30 14:42:10|     E 10 St & 2 Ave|2023-09-30 14:48:17|
|    E 1 St & Bowery|2023-09-12 01:18:36|Allen St & Riving...|2023-09-12 01:34:07|
|    E 1 St & Bowery|2023-09-13 22:11:19|Kingston Ave & Pa...|2023-09-13 22:39:51|
|    E 1 St & Bowery|2023-09-15 18:29:10|Greenwich Ave & C...|2023-09-15 18:47:04|
|    E 1 St & Bowery|2023-09-16 20:05:32|Christopher St & ...|2023-09-16 20:14:40|
|    E 1 St & Bowery|2023-09-02 16:17:55|     6 Ave & W 45 St|2023-09-02 17:40:30|
|    E 1 St & Bowery|2023-09-18 11:47:11|  Broadway & W 29 St|2023-09-18 12:02:46|
|   

Consulta SQL para obter o histórico de viagens da estação "E 1 St & Bowery"

In [44]:

historico_estacaoX = spark.sql("""
    SELECT
        *
    FROM
        dados_viagem
    WHERE
        nome_estacao_inicio = 'E 1 St & Bowery' OR nome_estacao_termino = 'E 1 St & Bowery'
""")

# Exibe o histórico de viagens 
historico_estacaoX.show()


+-------------------+-------------------+--------------------+-------------------+
|nome_estacao_inicio|          inicio_em|nome_estacao_termino|         termino_em|
+-------------------+-------------------+--------------------+-------------------+
|    E 1 St & Bowery|2023-09-03 10:20:41|     E 10 St & 2 Ave|2023-09-03 10:24:16|
|    E 1 St & Bowery|2023-09-19 13:40:48|     E 10 St & 2 Ave|2023-09-19 13:48:11|
|    E 1 St & Bowery|2023-09-30 14:42:10|     E 10 St & 2 Ave|2023-09-30 14:48:17|
|    E 1 St & Bowery|2023-09-12 01:18:36|Allen St & Riving...|2023-09-12 01:34:07|
|    E 1 St & Bowery|2023-09-13 22:11:19|Kingston Ave & Pa...|2023-09-13 22:39:51|
|    E 1 St & Bowery|2023-09-15 18:29:10|Greenwich Ave & C...|2023-09-15 18:47:04|
|    E 1 St & Bowery|2023-09-16 20:05:32|Christopher St & ...|2023-09-16 20:14:40|
|    E 1 St & Bowery|2023-09-02 16:17:55|     6 Ave & W 45 St|2023-09-02 17:40:30|
|    E 1 St & Bowery|2023-09-18 11:47:11|  Broadway & W 29 St|2023-09-18 12:02:46|
|   

06-Identificar a estação de bicicleta mais popular com base no número total de viagens iniciadas e exibir o resultado em uma tabela SQL

In [45]:
# Calcula o número total de viagens iniciadas em cada estação
estacoes_mais_populares = df_dados.groupBy("nome_estacao_inicio") \
    .count() \
    .withColumnRenamed("count", "total_viagens_iniciadas")

# Encontra a estação mais popular com base no número total de viagens iniciadas
estacao_mais_popular = estacoes_mais_populares.orderBy(col("total_viagens_iniciadas").desc()) \
    .limit(1)

# Exiba a estação mais popular
estacao_mais_popular.show()

+-------------------+-----------------------+
|nome_estacao_inicio|total_viagens_iniciadas|
+-------------------+-----------------------+
|    W 21 St & 6 Ave|                  13596|
+-------------------+-----------------------+



Consulta SQL a estação mais popular com base no número total de viagens iniciadas

In [46]:
estacao_mais_popular = spark.sql("""
    SELECT
        estacao,
        total_viagens_iniciadas
    FROM
        (
            SELECT
                nome_estacao_inicio AS estacao,
                COUNT(*) AS total_viagens_iniciadas
            FROM
                dados_viagem
            GROUP BY
                nome_estacao_inicio
        )
    ORDER BY
        total_viagens_iniciadas DESC
    LIMIT 1
""")

# Exibe a estação mais popular
estacao_mais_popular.show()

+---------------+-----------------------+
|        estacao|total_viagens_iniciadas|
+---------------+-----------------------+
|W 21 St & 6 Ave|                  13596|
+---------------+-----------------------+



07-Determinar os dias da semana com a maior e a menor quantidade de viagens e exibir os resultados em uma tabela SQL.

In [47]:
#Encontra o dia da semana mais popular
consulta_dia_mais_popular = df_dados.groupBy(date_format("inicio_em", "EEEE").alias("dia_semana_maior_qtd")) \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(1)

#Encontrar o dia da semana menos popular
consulta_dia_menos_popular = df_dados.groupBy(date_format("inicio_em", "EEEE").alias("dia_semana_menor_qtd")) \
    .count() \
    .orderBy(col("count")) \
    .limit(1)

# Exibe Resultado
resultado_dia_mais_popular = consulta_dia_mais_popular.show()

# Exibe Resultado
resultado_dia_menos_popular = consulta_dia_menos_popular.show()


+--------------------+------+
|dia_semana_maior_qtd| count|
+--------------------+------+
|              Friday|609672|
+--------------------+------+

+--------------------+------+
|dia_semana_menor_qtd| count|
+--------------------+------+
|              Monday|336507|
+--------------------+------+



Consulta SQL para encontrar o dia da semana mais popular

In [48]:

consulta01 = spark.sql("""
SELECT
    DATE_FORMAT(inicio_em, 'EEEE') AS dia_semana_maior_qtd,
    COUNT(*) AS quantidade_viagens
FROM dados_viagem
GROUP BY DATE_FORMAT(inicio_em, 'EEEE')
ORDER BY quantidade_viagens DESC
LIMIT 1
""")

# Consulta SQL para encontrar o dia da semana menos popular
consulta02 = spark.sql("""
SELECT
    DATE_FORMAT(inicio_em, 'EEEE') AS dia_semana_menor_qtd,
    COUNT(*) AS viagens_dia_menos_popular
FROM dados_viagem
GROUP BY DATE_FORMAT(inicio_em, 'EEEE')
ORDER BY viagens_dia_menos_popular
LIMIT 1
""")
# Executar a consulta para encontrar o dia da semana mais popular
resultado_dia_mais_popular = consulta01.show()

# Executar a consulta para encontrar o dia da semana menos popular
resultado_dia_menos_popular = consulta02.show()

+--------------------+------------------+
|dia_semana_maior_qtd|quantidade_viagens|
+--------------------+------------------+
|              Friday|            609672|
+--------------------+------------------+

+--------------------+-------------------------+
|dia_semana_menor_qtd|viagens_dia_menos_popular|
+--------------------+-------------------------+
|              Monday|                   336507|
+--------------------+-------------------------+



In [49]:
# Encerra a sessão Spark
spark.stop()